In [ ]:
import os
import sys

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [ ]:
%env JAX_DEBUG_NANS=False

from collections import namedtuple

import numpy as np
import jax
import jax.numpy as jnp
from jax import jit, vmap, grad, value_and_grad
from jax.experimental.optimizers import adam, nesterov, clip_grads

import haiku as hk

import oryx.core as core

import matplotlib.pyplot as plt
import pickle

from sdrf import (
    extract_debug,
    render,
    render_img,
    run_one_iter_of_sdrf,
    eikonal_loss,
    manifold_loss,
    SDRF,
    SDRFParams,
    Siren
)
from util import get_ray_bundle, look_at, get_bunny, sdf_mesh

In [ ]:
def analytic_geometry(x, params):
    origin, radius = params
    dumb_norm = lambda a, b: ((a - b) ** 2).sum() ** (1 / 2)
    return (dumb_norm(x, origin) - radius)

In [ ]:
def analytic_appearance(pt, rd, params):
    '''origin, radius = params
    light_pos = jnp.array([-8.0, -4.0, 0.0])
    normalize = lambda vec: vec / jnp.linalg.norm(vec, ord=2)
    #normals = lambda pt: grad(lambda pt, params: analytic_geometry(pt, params).sum(), argnums=(0,))(
    #    pt, (origin, radius)
    #)[0]
    
    error = jnp.abs(analytic_geometry(pt, params))
    valid_mask = error < 1e-2
    
    masked_sdf = lambda pt, params: jax.lax.cond(valid_mask.sum(), 
                                                 (pt, params), 
                                                 lambda args: jnp.array(analytic_geometry(*args)).reshape(1),
                                                 jnp.zeros(1,), 
                                                 lambda x: x)
    normals = lambda pt: grad(lambda pt, params: masked_sdf(pt, params).sum(), argnums=(0,))(
        pt, (origin, radius)
    )[0]
    
    distance = lambda pt: jnp.square(jnp.linalg.norm(light_pos - pt, ord=2))
    light_dir = lambda pt: normalize(light_pos - pt)
    diffuse_power = 20.0
    specular_power = 40.0
    specular_hardness = 16
    diffuse = lambda pt: jnp.broadcast_to(
        jnp.clip(
            jnp.dot(light_dir(pt), normals(pt)) * diffuse_power / distance(pt), 0.0, 1.0
        ),
        (3,),
    )
    h = lambda pt, rd: normalize(light_dir(pt) - rd)
    ndoth = lambda pt, rd: jnp.clip(jnp.dot(normals(pt), h(pt, rd)), 0.0, 1.0)
    specular = (
        lambda pt, rd: jnp.power(ndoth(pt, rd), specular_hardness)
        * specular_power
        / distance(pt)
    )
    
    return diffuse(pt) + specular(pt, rd)'''
    return jnp.array(params[0])
    #return jnp.array([1.0, 1.0, 1.0])

In [ ]:
RenderOptions = namedtuple(
    "RenderOptions", 
    ["phi", "num_samples", "truncation_distance", "additive", "chunksize", "isosurfaces_debug", "oryx_debug"]
)
PhiOptions = namedtuple("PhiOptions", ["initial_sigma", "lr_decay", "lr_decay_factor"])
SDRFOptions = namedtuple("SDRFOptions", ["eikonal", "manifold", "sampler", "render"])
AuxLossOptions = namedtuple("AuxLossOptions", ["scale", "num_samples"])
#SamplerOptions = namedtuple("SamplerOptions", ["kind", "gaussian"])
#SamplerOptions = namedtuple("SamplerOptions", ["kind", "linear"])
SamplerOptions = namedtuple("Stratified", ["kind", "stratified"])
GaussianSamplerOptions = namedtuple("GaussianSamplerOptions", ["sigma"])
LinearSamplerOptions = namedtuple("LinearSamplerOptions", ["support"])
StratifiedSamplerOptions = namedtuple("StratifiedSamplerOptions", ["support"])

In [ ]:
def visualize_gradient(g):
    if len(g.shape) > 2:
        min_val, max_val = jnp.amin(g, axis=(0, 1)), jnp.amax(g, axis=(0, 1))
    else:
        min_val, max_val = jnp.min(g.ravel()), jnp.max(g.ravel())
    
    scaled = (g - min_val) / (max_val - min_val + 1e-9)
    
    plt.imshow(scaled)
    plt.show()

In [ ]:
%matplotlib inline
def optimization_experiment(appearance_fn, geometry_fn, sdrf_params, num_iters=1000, lr=1e-3):
    rng = jax.random.PRNGKey(42)

    sdrf = SDRF(geometry=geometry_fn, appearance=appearance_fn)
    #sdrf_params = SDRFParams(geometry=(origins, radius), appearance=(origins, radius))

    '''init_adam, update, get_params = adam(
        lambda iteration: lr
    )'''
    init_adam, update, get_params = nesterov(
        lambda iteration: lr, 1.0
    )
    
    optimizer_state = init_adam(sdrf_params)
    
    view_matrix = jnp.array(
        np.linalg.inv(
            np.array(
                look_at(
                    jnp.array([-4.0, 0.0, 0.0]),
                    jnp.array([0.0, 0.0, 0.0]),
                    jnp.array([0.0, 1.0, 0.0]),
                )
            )
        )
    )

    height, width, chunk_size = 256, 256, 2 ** 14
    uv, ro, rd = get_ray_bundle(height, width, 100.0, view_matrix)
    
    #phi_options = PhiOptions(initial_sigma=1e-1, lr_decay_factor=0.1, lr_decay=250)
    phi_options = PhiOptions(initial_sigma=5e-2, lr_decay_factor=0.1, lr_decay=250)
    render_options = RenderOptions(
        phi=phi_options,
        num_samples=8, 
        truncation_distance=1.0,
        additive=False,
        chunksize=16192,
        isosurfaces_debug=True,
        oryx_debug=True
    )
    #sampler_options = SamplerOptions(kind="gaussian", gaussian=GaussianSamplerOptions(sigma=1e-2))
    #sampler_options = SamplerOptions(kind="linear", linear=LinearSamplerOptions(support=5e-2))
    sampler_options = SamplerOptions(kind="stratified", stratified=StratifiedSamplerOptions(support=5e-2))
    options = SDRFOptions(
        eikonal=AuxLossOptions(scale=4.0, num_samples=1024),
        manifold=AuxLossOptions(scale=4.0, num_samples=1024),
        render=render_options,
        sampler=sampler_options
    )
    
    def get_target_img(target_params, subrng):
        analytic_sdrf = SDRF(appearance=analytic_appearance, geometry=analytic_geometry)
        target_s, _, _ = run_one_iter_of_sdrf(analytic_sdrf, target_params, uv, ro, rd, 1, options, subrng)
        
        #mesh_sdrf = SDRF(appearance=analytic_appearance, geometry=sdf_mesh)
        #target_s, _, _ = run_one_iter_of_sdrf(mesh_sdrf, target_params, ro, rd, 1, options, subrng)
        
        return target_s
    
    #with jax.disable_jit():
    rng, subrng = jax.random.split(rng)
    
    # for analytic case
    target_geometry_params = (jnp.zeros((3,)), jnp.array([2.0]))
    target_appearance_params = (jnp.array([1.0, 1.0, 1.0]),)
    target_s = get_target_img(SDRFParams(geometry=target_geometry_params, 
                                         appearance=target_appearance_params), 
                              subrng)
    
    # for mesh case
    #mesh = jnp.array(get_bunny())
    #target_s = get_target_img(SDRFParams(geometry=mesh, appearance=mesh), subrng)
    
    #print(target_s.flatten().max())
    
    plt.imshow(np.array(target_s).reshape(height, width, 3))
    plt.show()

    def loss_fn(params, iteration, subrng):
        rgb, depth, debug = run_one_iter_of_sdrf(sdrf, params, uv, ro, rd, iteration, options, subrng[0])
        
        eikonal_samples = (
            jax.random.uniform(subrng[1], (options.eikonal.num_samples, 3))
            * options.eikonal.scale
        )

        manifold_samples = (
            jax.random.uniform(subrng[2], (options.manifold.num_samples, 3))
            * options.manifold.scale
        )
        
        rgb_loss = jnp.mean(((target_s - rgb) ** 2.0).flatten())
        
        e_loss, m_loss = (
            eikonal_loss(sdrf.geometry, eikonal_samples, params.geometry),
            manifold_loss(sdrf.geometry, manifold_samples, params.geometry).sum(),
        )
        
        losses = jnp.array([rgb_loss, e_loss, m_loss])
        loss_weights = jnp.array([3e3, 5e1, 1e2])
        #loss_weights = jnp.array([1.0, 1.0, 1.0])
        #loss_weights = jnp.array([1.0, 1e-9, 1e-9])
        
        return jnp.dot(losses, loss_weights), (losses, rgb, depth, debug)
    
    value_and_grad_fn = jit(value_and_grad(loss_fn, argnums=(0,), has_aux=True))
    reap_fn = jit(core.reap(value_and_grad(loss_fn, argnums=(0,), has_aux=True), tag="vjp"))
    
    for i in range(num_iters):
        rng, *subrng = jax.random.split(rng, 5)
        params = get_params(optimizer_state)
        #params = SDRFParams(appearance=(jnp.array([1.0, 1.0, 1.0]),), geometry=params.geometry)
        print(params.appearance)
        
        #with jax.disable_jit():
        (loss, (losses, rgb, depth, dists)), (grads,) = value_and_grad_fn(
             params, i, subrng
        )
        grads = clip_grads(grads, 1.0)
        
        optimizer_state = update(i, grads, optimizer_state)
        
        if i % 100 == 0:
            debug_uv = reap_fn(params, i, subrng)
            debug_reshaped = extract_debug(debug_uv['uv'], debug_uv['dvsdattr_0'][:, :, 3, :], height, width)
            #debug_reshaped = extract_debug(debug_uv['uv'], debug_uv['debug_0'], height, width)
        
            visualize_gradient(debug_reshaped)
            
            plt.imshow(rgb.reshape(height, width, 3))
            plt.show()
            for j in range(8):
                plt.imshow(dists.reshape(height, width, 8)[:, :, j], cmap='gray')
                plt.show()
        
        print(f"Iteration {i}: Loss: {loss}\n\tRGB loss: {losses[0]}\n\tEikonal loss: {losses[1]}\n\tManifold loss: {losses[2]}")
        #print(f"\tRadius: {params.geometry[1]}\n")

In [ ]:
'''origins, radius = jnp.zeros((3,)), 1.0
color = jnp.array([0.0, 1.0, 0.0])
sdrf_params = SDRFParams(geometry=(origins, radius), appearance=(color,))
optimization_experiment(analytic_appearance, analytic_geometry, sdrf_params, num_iters=10000, lr=1e-3)'''

In [ ]:
color = jnp.array([1.0, 0.5, 0.5])
with open("sphere_nerf.pkl", "rb") as pkl:
    sdrf_params = SDRFParams(
        geometry=pickle.load(pkl), appearance=(color,)
    )

model_fn = hk.transform(lambda x: Siren(3, 1, 4, 32, False, "relu")(x))
model_fn = hk.without_apply_rng(model_fn)

optimization_experiment(analytic_appearance, 
                        lambda pt, params: model_fn.apply(params, pt).sum(), 
                        sdrf_params, 
                        lr=2e-4)